In [1]:
from decouple import config
import intrinio_sdk
from intrinio_sdk.rest import ApiException
import numpy as np 
import pandas as pd
import re
import string

In [47]:
import datetime
import calendar

def add_three_months(orig_date):
    # advance year and month by one month
    new_year = orig_date.year
    new_month = orig_date.month + 3
    # note: in datetime.date, months go from 1 to 12
    if new_month > 12:
        new_year += 1
        new_month -= 12

    last_day_of_month = calendar.monthrange(new_year, new_month)[1]
    new_day = min(orig_date.day, last_day_of_month)

    return orig_date.replace(year=new_year, month=new_month, day=new_day)

In [48]:
def get_fundamentals(tkr_id, n_toget, sandbox=False, return_df=False):
    if sandbox == False:
        intrinio_sdk.ApiClient().configuration.api_key['api_key'] = config('INTRINIO_KEY')

        security_api = intrinio_sdk.SecurityApi()
    elif sandbox == True:
        intrinio_sdk.ApiClient().configuration.api_key['api_key'] = config('INTRINIO_SANDBOX_KEY')

        security_api = intrinio_sdk.SecurityApi()

    
    capi = intrinio_sdk.CompanyApi()
    fapi = intrinio_sdk.FundamentalsApi()

    fund_params = {
        'identifier'     :tkr_id,
        'filed_after'    :'',
        'filed_before'   :'',
        'reported_only'  : False,
    #     'fiscal_year'  :fiscal_year,
        'statement_code' :'',
        'type'           :'',
        'start_date'     :'',
        'end_date'       :'',
        'page_size'      :n_toget,
        'next_page'      :''
    }
    
    fundamentals = capi.get_company_fundamentals(**fund_params)
    id_dict = {
        "start_date"     : [],
        "end_date"       : [],
        "fiscal_year"    : [],
        "fiscal_period"  : [],
        "id"             : [],
        "statement_code" : [],
    }

    for i in np.arange(len(fundamentals.fundamentals)):
        id_dict['start_date'].append(fundamentals.fundamentals[i].start_date)
        id_dict['end_date'].append(fundamentals.fundamentals[i].end_date) 
        id_dict['fiscal_year'].append(fundamentals.fundamentals[i].fiscal_year)
        id_dict['fiscal_period'].append(fundamentals.fundamentals[i].fiscal_period)
        id_dict['id'].append(fundamentals.fundamentals[i].id)
        id_dict['statement_code'].append(fundamentals.fundamentals[i].statement_code)

    id_df = pd.DataFrame.from_dict(id_dict)
    
    qtrs = ['Q1', 'Q2', 'Q3', 'Q4']
    income_statements = (id_df.loc[(id_df['statement_code'] == 'income_statement') &
                                   (id_df['fiscal_period'].isin(qtrs)==True)]
                         .sort_values(by='start_date'))
    
    fund_dict = {}
    for row in income_statements.iterrows():
        fundamentals_ret = fapi.get_fundamental_standardized_financials(row[1]['id'])
        fund_get = fundamentals_ret.standardized_financials_dict
        fund_info = fundamentals_ret.fundamental_dict
        funds = {}
        funds['date'] = fund_info['filing_date']
        funds['fiscal_year'] = fund_info['fiscal_year']
        funds['quarter'] = fund_info['fiscal_period']
        for f in fund_get:
#             fs = {
#                 'unit':f['data_tag']['unit'],
#                 'value':f['value']
#             }
            funds[f['data_tag']['tag'].lower()] = f['value']
        fund_dict[row[1]['id']] = funds
        
    if return_df == False:
        return(fund_dict)
    elif return_df == True:
        temp_dict = {}
        for fun in fund_dict[list(fund_dict.keys())[0]].keys():
            temp_dict[fun] = []
        
        for fun_key in fund_dict.keys():
            for fun in temp_dict.keys():
                if fun != 'date':
                    temp_dict[fun].append(fund_dict[fun_key][fun]
                                           if fun in list(fund_dict[fun_key].keys()) 
                                           else 0)
                elif fun == 'date':
                    temp_dict[fun].append(fund_dict[fun_key][fun]
                                           if fund_dict[fun_key]['date'] != None
                                           else add_three_months(previous_date))
                    previous_date = fund_dict[fun_key]['date']
        return_df = pd.DataFrame(temp_dict)
        return(return_df)
        

In [49]:
get_fundamentals('TSLA', 100, return_df=True)

,date,fiscal_year,quarter,operatingrevenue,totalrevenue,operatingcostofrevenue,totalcostofrevenue,totalgrossprofit,sgaexpense,rdexpense,...,incometaxexpense,netincomecontinuing,netincome,netincometocommon,weightedavebasicsharesos,basiceps,weightedavedilutedsharesos,dilutedeps,weightedavebasicdilutedsharesos,basicdilutedeps
0,2017-11-02 19:56:36+00:00,2016.0,Q3,2.298436e+09,2.298436e+09,1.661701e+09,1.661701e+09,6.367350e+08,336811000.0,214302000.0,...,8133000.0,21878000.0,21878000.0,21878000.0,148991000.0,0.15,156935000.0,0.14,145900000.0,0.15
1,2018-02-02 19:56:36+00:00,2016.0,Q4,2.284631e+09,2.284631e+09,1.849353e+09,1.849353e+09,4.352780e+08,456016000.0,245960000.0,...,11070000.0,-219469000.0,-219469000.0,-121337000.0,144212000.0,-0.61,144212000.0,-0.60,144200000.0,-0.61
2,2018-05-04 21:46:25+00:00,2017.0,Q1,2.696270e+09,2.696270e+09,2.028324e+09,2.028324e+09,6.679460e+08,603455000.0,322040000.0,...,25278000.0,-397181000.0,-397181000.0,-330277000.0,162129000.0,-2.04,162129000.0,-2.04,161900000.0,-2.04
3,2018-08-03 21:56:44+00:00,2017.0,Q2,2.789557e+09,2.789557e+09,2.122942e+09,2.122942e+09,6.666150e+08,537757000.0,369774000.0,...,15647000.0,-401427000.0,-401427000.0,-336397000.0,165212000.0,-2.04,165212000.0,-2.04,164900000.0,-2.04
4,2018-11-01 20:38:56+00:00,2017.0,Q3,2.984675e+09,2.984675e+09,2.535535e+09,2.535535e+09,4.491400e+08,652998000.0,331622000.0,...,-285000.0,-671163000.0,-671163000.0,-619376000.0,167294000.0,-3.70,167294000.0,-3.70,167400000.0,-3.70
5,2019-02-01 20:38:56+00:00,2017.0,Q4,3.288249e+09,3.288249e+09,2.849463e+09,2.849463e+09,4.387860e+08,682290000.0,354637000.0,...,-9094000.0,-770807000.0,-770807000.0,-675350000.0,165758000.0,-4.05,165758000.0,-4.05,165800000.0,-4.05
6,2019-04-29 06:09:07+00:00,2018.0,Q1,3.408751e+09,3.408751e+09,2.952225e+09,2.952225e+09,4.565260e+08,686404000.0,367096000.0,...,5605000.0,-784627000.0,-784627000.0,-709551000.0,169146000.0,-4.19,169146000.0,-4.19,169300000.0,-4.19
7,2019-07-29 06:21:00+00:00,2018.0,Q2,4.002231e+09,4.002231e+09,3.383301e+09,3.383301e+09,6.189300e+08,750759000.0,386129000.0,...,13707000.0,-742706000.0,-742706000.0,-717539000.0,169997000.0,-4.22,169997000.0,-4.22,170000000.0,-4.22
8,2018-11-01 20:38:56+00:00,2018.0,Q3,6.824413e+09,6.824413e+09,5.300748e+09,5.300748e+09,1.523665e+09,729876000.0,350848000.0,...,16647000.0,254673000.0,254673000.0,311516000.0,170893000.0,1.82,178196000.0,1.75,171200000.0,1.82
9,2019-02-01 20:38:56+00:00,2018.0,Q4,7.225873e+09,7.225873e+09,5.782973e+09,5.782973e+09,1.442900e+09,667452000.0,356297000.0,...,21878000.0,210078000.0,210078000.0,139483000.0,170525000.0,0.87,170525000.0,0.94,170600000.0,0.87


In [7]:
# This function will return a dictionary of company names and symbol
# or abbreviation. Don't run this on non-sandbox api unless you
# want a huge list.

def get_company_info(id_type='', sandbox=False):
    
    if sandbox == False:
        intrinio_sdk.ApiClient().configuration.api_key['api_key'] = config('INTRINIO_KEY')

        security_api = intrinio_sdk.SecurityApi()
    elif sandbox == True:
        intrinio_sdk.ApiClient().configuration.api_key['api_key'] = config('INTRINIO_SANDBOX_KEY')

        security_api = intrinio_sdk.SecurityApi()
    
    
    """ticker, name, lei, id, cik"""
    capi = intrinio_sdk.CompanyApi()
    companies = capi.get_all_companies().companies_dict
    names_dict = {}
    for company in companies:
        names_dict[company['name']] = company[id_type]
    return(names_dict)

NameError: name 'relativedelta' is not defined